<a href="https://colab.research.google.com/github/md-junaid79/MultiAgent-with-Autogen-/blob/main/Multiagent_system_using_autogen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.0/834.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.0 MB/s eta 0:00:00


In [ ]:
!pip install pyautogen

In [ ]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
import autogen

Patching name='__init__', member=<function LLMLingua.__init__ at 0x7bfd57b01bc0>, patched=<function function.__call__ at 0x7bfd57b01b20>
Patching name='compress_text', member=<function LLMLingua.compress_text at 0x7bfd57b01c60>, patched=<function function.__call__ at 0x7bfd57b01ee0>


In [ ]:
from autogen import AssistantAgent, UserProxyAgent

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
config_list = [{'model': 'gemini-2.5-flash', 'api_key':GOOGLE_API_KEY}]

In [ ]:
llm_config={"config_list": config_list}

In [ ]:
gpt4o_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0.3,
    "config_list": config_list,
    "timeout": 120,
}


In [ ]:
task = '''
 **Task**: As an architect, you are required to design a solution for the
 following business requirements:
    - Data storage for massive amounts of IoT data
    - Real-time data analytics and machine learning pipeline
    - Scalability
    - Cost Optimization
    - Region pairs in Europe, for disaster recovery
    - Tools for monitoring and observability
    - Timeline: 6 months

    Break down the problem using a Chain-of-Thought approach. Ensure that your
    solution architecture is following best practices.
    '''

In [ ]:
cloud_prompt = '''
**Role**: You are an expert cloud architect. You need to develop architecture proposals
using either cloud-specific PaaS services, or cloud-agnostic ones.
The final proposal should consider all 3 main cloud providers: Azure, AWS and GCP, and provide
a data architecture for each. At the end, briefly state the advantages of cloud over on-premises
architectures, and summarize your solutions for each cloud provider using a table for clarity.
'''
cloud_prompt += task

In [ ]:
oss_prompt = '''
**Role**: You are an expert on-premises, open-source software architect. You need
to develop architecture proposals without considering cloud solutions.
 Only use open-source frameworks that are popular and have lots of active contributors.
 At the end, briefly state the advantages of open-source adoption, and summarize your
 solutions using a table for clarity.
'''
oss_prompt += task

In [ ]:
lead_prompt =  '''
**Role**: You are a lead Architect tasked with managing a conversation between
the cloud and the open-source Architects.
Each Architect will perform a task and respond with their resuls. You will critically
review those and also ask for, or point to, the disadvantages of their solutions.
You will review each result, and choose the best solution in accordance with the business
requirements and architecture best practices. You will use any number of summary tables to
communicate your decision.
'''
lead_prompt += task

In [ ]:
user_proxy=UserProxyAgent(
    name="supervisor",
    system_message = "A Human Head of Architecture",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False,
    },
    human_input_mode="NEVER",


)

In [ ]:
cloud_agent = AssistantAgent(
    name = "cloud",
    system_message = cloud_prompt,
    llm_config={"config_list": config_list}
    )

oss_agent = AssistantAgent(
    name = "oss",
    system_message = oss_prompt,
    llm_config={"config_list": config_list}
    )

lead_agent = AssistantAgent(
    name = "lead",
    system_message = lead_prompt,
    llm_config={"config_list": config_list}
)

In [ ]:
def state_transition(last_speaker,groupchat):
   messages = groupchat.messages

   if last_speaker is user_proxy:
       return cloud_agent
   elif last_speaker is cloud_agent:
       return oss_agent
   elif last_speaker is oss_agent:
       return lead_agent
   elif last_speaker is lead_agent:
       # lead -> end
       return None

supervisoragent-->cloudagent-->ossagent-->leadagent

In [ ]:
groupchat=autogen.GroupChat(
    agents=[user_proxy, cloud_agent, oss_agent, lead_agent],
    messages=[],
    max_round=6,
    speaker_selection_method=state_transition,
)

In [ ]:
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(
    manager, message="Provide your best architecture based on the AI agent for the business requirements."
)

supervisor (to chat_manager):

Provide your best architecture based on the AI agent for the business requirements.

--------------------------------------------------------------------------------

Next speaker: cloud

cloud (to chat_manager):

To design an architecture for the specified business requirements, we will break down the problem into its core components, consider each requirement, and then propose solutions for Azure, AWS, and GCP.

### Chain-of-Thought Breakdown:

1. **Data Storage for Massive Amounts of IoT Data**:
   - IoT data typically requires a solution that can handle large-scale, high-velocity data ingestion with low latency.
   - Data lakes or time-series databases optimized for IoT data are suitable options.

2. **Real-time Data Analytics and Machine Learning Pipeline**:
   - A streaming data processing service is needed for real-time analytics.
   - Machine learning services will require data preparation, model training, and serving capabilities.

3. **Scalabili

ChatResult(chat_id=None, chat_history=[{'content': 'Provide your best architecture based on the AI agent for the business requirements.', 'role': 'assistant', 'name': 'supervisor'}, {'content': "To design an architecture for the specified business requirements, we will break down the problem into its core components, consider each requirement, and then propose solutions for Azure, AWS, and GCP.\n\n### Chain-of-Thought Breakdown:\n\n1. **Data Storage for Massive Amounts of IoT Data**:\n   - IoT data typically requires a solution that can handle large-scale, high-velocity data ingestion with low latency.\n   - Data lakes or time-series databases optimized for IoT data are suitable options.\n\n2. **Real-time Data Analytics and Machine Learning Pipeline**:\n   - A streaming data processing service is needed for real-time analytics.\n   - Machine learning services will require data preparation, model training, and serving capabilities.\n\n3. **Scalability**:\n   - We need an infrastructure 

In [ ]:
user_proxy.initiate_chat(
    manager, message="i have a business problem with education industry we are not able to get proper assistant can you create one sclable chatbot architecture."
)

supervisor (to chat_manager):

i have a business problem with education industry we are not able to get proper assistant can you create one sclable chatbot architecture.

--------------------------------------------------------------------------------

Next speaker: cloud

cloud (to chat_manager):

Creating a scalable chatbot architecture for the education industry involves designing a system that can handle queries from numerous users, provide meaningful responses, and continuously evolve with user interactions. Here's a step-by-step breakdown using a Chain-of-Thought approach:

### 1. Understand the Requirements:
   - **Educational Domain Specific**: The chatbot must understand educational terms, queries, and contexts.
   - **Scalability**: The system should support a large number of concurrent users.
   - **Machine Learning**: It should learn from interactions to improve responses over time.
   - **Cost-efficient**: The solution must optimize costs.
   - **High Availability and Disa

ChatResult(chat_id=None, chat_history=[{'content': 'i have a business problem with education industry we are not able to get proper assistant can you create one sclable chatbot architecture.', 'role': 'assistant', 'name': 'supervisor'}, {'content': "Creating a scalable chatbot architecture for the education industry involves designing a system that can handle queries from numerous users, provide meaningful responses, and continuously evolve with user interactions. Here's a step-by-step breakdown using a Chain-of-Thought approach:\n\n### 1. Understand the Requirements:\n   - **Educational Domain Specific**: The chatbot must understand educational terms, queries, and contexts.\n   - **Scalability**: The system should support a large number of concurrent users.\n   - **Machine Learning**: It should learn from interactions to improve responses over time.\n   - **Cost-efficient**: The solution must optimize costs.\n   - **High Availability and Disaster Recovery**: Regional pairings in Europ